# Mitochondria removal protocol


This tutorial will cover how to use the preconstructed extended taxonomies from Sonett et al. to remove mitochondria from 16S rRNA gene sequence data in QIIME2.

If you need to build your own extended taxonomy, see the extended [taxonomy creation tutorial here](extended_taxonomy_construction_tutorial.ipynb)

The tutorial assumes that you have a top-level analysis folder, and within it an `input` `output` and `procedure` folder, and that this script is located in the procedure folder. Therefore, whenever we refer to files in input it will be with relative paths like `../input/name_of_some_file.txt`. You can either create these in your finder, or on the command line. The exact command will vary by command line interface, but on BASH or PowerShell you can type `mkdir input` to create an input directory in your current folder, `mkdir output` to create an output directory in your current folder, etc.

It's further assumed that QIIME2 is installed.

Given all that, this tutorial will discuss how to use the the supplemented databases to remove mitochondria from your 16S datasets using Qiime2. 

# Classify taxonomy with vsearch 

### Set up the directories and import files for this part of the analysis

In [4]:
from os.path import join, abspath, exists
working_dir = abspath('..')
input_dir = join(working_dir,'input')
output_dir = join(working_dir, 'output')


#Define variables to hold the filepaths for metadata, sequences, and the extended taxonomy reference

#these should be adjusted to point to your data
metadata_file_name = 'sample_metadata_live_vs_dead_combo.tsv'
sequence_file_name = 'rep_seqs_merged.qza'
feature_table_name = 'feature_table_live_vs_dead.qza'

metadata_path = join(input_dir, metadata_file_name)
seqs_path = join(input_dir, sequence_file_name)
feature_table_path = join(input_dir, feature_table_name)

#these will be automaticallly downloaded if not found
taxonomy_reference_dir = join(input_dir,'taxonomy_references')
base_silva_paths = [join(taxonomy_reference_dir, 'silva_sequences.qza'),
                    join(taxonomy_reference_dir, 'silva_taxonomy.qza')]
extended_silva_paths = [join(taxonomy_reference_dir, 'silva_extended_sequences.qza'),
                        join(taxonomy_reference_dir, 'silva_extended_taxonomy.qza')]

### Verify that all files exist

In [17]:
import os
import shutil
import urllib.request

def download_file(url, local_filepath):
    with urllib.request.urlopen(url) as response, open(local_filepath, 'wb') as out_file:
        shutil.copyfileobj(response, out_file)

this is being troublesome ... thoughts?
https://stackoverflow.com/questions/34692009/download-image-from-url-using-python-urllib-but-receiving-http-error-403-forbid

In [23]:
print("Verifying that all needed starting data files exist.")

### Add file names for the actual files we need to the required files list

required_filepaths = ([taxonomy_reference_dir, metadata_path, seqs_path, feature_table_path]
                      + base_silva_paths + extended_silva_paths)

for existing_file in required_filepaths:
    if not exists(existing_file):
        if existing_file == taxonomy_reference_dir:
            os.mkdir(taxonomy_reference_dir)
        elif existing_file in base_silva_paths:
            download_file('https://data.qiime2.org/2021.4/common/silva-138-99-seqs-515-806.qza',
                          join(taxonomy_reference_dir, 'silva_sequences.qza'))
            download_file('https://data.qiime2.org/2021.4/common/silva-138-99-tax-515-806.qza', 
                          join(taxonomy_reference_dir, 'silva_taxonomy.qza'))
        elif existing_file in extended_silva_paths:
            download_file('https://zenodo.org/records/10251912/files/silva_extended_sequences.qza?download=1',
                          join(taxonomy_reference_dir, 'silva_extended_sequences.qza'))
            download_file('https://zenodo.org/records/10251912/files/silva_extended_taxonomy.qza?download=1',
                          join(taxonomy_reference_dir, 'silva_extended_taxonomy.qza'))            
        else:
            raise IOError(f"Required file {existing_file} not found. Please ensure it is in that directory.")
        
    print(f"{existing_file}.....OK")
print("Done.")

Verifying that all needed starting data files exist.
/mnt/c/Users/dsone/Documents/zaneveld/organelle_removal/organelle_removal/Tutorial/input/taxonomy_references.....OK
/mnt/c/Users/dsone/Documents/zaneveld/organelle_removal/organelle_removal/Tutorial/input/sample_metadata_live_vs_dead_combo.tsv.....OK
/mnt/c/Users/dsone/Documents/zaneveld/organelle_removal/organelle_removal/Tutorial/input/rep_seqs_merged.qza.....OK
/mnt/c/Users/dsone/Documents/zaneveld/organelle_removal/organelle_removal/Tutorial/input/taxonomy_references/silva_sequences.qza.....OK
/mnt/c/Users/dsone/Documents/zaneveld/organelle_removal/organelle_removal/Tutorial/input/taxonomy_references/silva_taxonomy.qza.....OK


HTTPError: HTTP Error 403: Forbidden

# Annotate sequences

We will use vsearch to annotate taxonomy. This will be done once for each of the refernce taxonomies :  Silva, and Silva + MeTaxa2 + phytoref reference mitocondrial sequences. 

## Load metadata and sequence files as .qza artifacts

Next we'll load your study-specific metadata and sequence files as QIIME2 Artifacts

In [50]:
metadata = Metadata.load(metadata_path)
seqs = Artifact.load(seqs_path)

## Use VSEARCH to classify your sequences according to base and extended SILVA taxonomies

**Note:** On a full dataset this step can take a while to run. Adjusting the vsearch threads parameter can help speed up the process if you have enough memory to support multiple threads (about 8GB / thread is recommended)

see https://forum.qiime2.org/t/vsearch-classifier-memory/8667/5

In [51]:
references = ['silva','silva_extended']

vsearch_results = {}
for reference in references:
    
    #Note: The next two lines just set paths to the sequence and taxonomy qza files, respectively
    #If using a custom reference set, you could either name it with the same naming scheme 
    # my_reference_sequences.qza and my_reference_taxonomy.qza, and add 'my_reference' to the 
    #references list up above, or just manually set the file names using this section as a 
    #loose guide.
    
    reference_otu_path = join(taxonomy_reference_dir,f'{reference}_sequences.qza')
    reference_taxonomy_path = join(taxonomy_reference_dir,f'{reference}_taxonomy.qza')
    
    #Load .qza files as QIIME2 artifacts
    reads = Artifact.load(reference_otu_path)
    taxonomy = Artifact.load(reference_taxonomy_path)
    
    #Run VSEARCH, and store results in the vsearch_results dictionary
    vsearch_results[reference] = classify_consensus_vsearch(seqs, reads, taxonomy, threads = 4)
    

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: vsearch --usearch_global /tmp/qiime2-archive-6ri4ri5f/d7bb1a41-d3e8-465f-be11-90b58e1cf211/data/dna-sequences.fasta --id 0.8 --query_cov 0.8 --strand both --maxaccepts 10 --maxrejects 0 --db /tmp/qiime2-archive-5j06xdmi/9d299643-4922-44bc-8003-3d5185d76805/data/dna-sequences.fasta --threads 4 --output_no_hits --blast6out /tmp/tmpirqojtnn



vsearch v2.7.0_linux_x86_64, 15.5GB RAM, 8 cores
https://github.com/torognes/vsearch

Reading file /tmp/qiime2-archive-5j06xdmi/9d299643-4922-44bc-8003-3d5185d76805/data/dna-sequences.fasta 100%
51405917 nt in 202865 seqs, min 81, max 1003, avg 253
Masking 100%
Counting k-mers 100%
Creating k-mer index 100%
Searching 100%
Matching query sequences: 0 of 464 (0.00%)


Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: vsearch --usearch_global /tmp/qiime2-archive-6ri4ri5f/d7bb1a41-d3e8-465f-be11-90b58e1cf211/data/dna-sequences.fasta --id 0.8 --query_cov 0.8 --strand both --maxaccepts 10 --maxrejects 0 --db /tmp/qiime2-archive-mjgxjjvw/b41681fb-a4e7-4ef8-a23a-a26f1bcfd272/data/dna-sequences.fasta --threads 4 --output_no_hits --blast6out /tmp/tmpqgopdrng



vsearch v2.7.0_linux_x86_64, 15.5GB RAM, 8 cores
https://github.com/torognes/vsearch

Reading file /tmp/qiime2-archive-mjgxjjvw/b41681fb-a4e7-4ef8-a23a-a26f1bcfd272/data/dna-sequences.fasta 100%
86453445 nt in 313734 seqs, min 54, max 2366, avg 276
Masking 100%
Counting k-mers 100%
Creating k-mer index 100%
Searching 100%
Matching query sequences: 127 of 464 (27.37%)


Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: vsearch --usearch_global /tmp/qiime2-archive-6ri4ri5f/d7bb1a41-d3e8-465f-be11-90b58e1cf211/data/dna-sequences.fasta --id 0.8 --query_cov 0.8 --strand both --maxaccepts 10 --maxrejects 0 --db /tmp/qiime2-archive-ewumxl2w/327da406-e7db-4898-931f-df01072fcefd/data/dna-sequences.fasta --threads 4 --output_no_hits --blast6out /tmp/tmpr3s8ims5



vsearch v2.7.0_linux_x86_64, 15.5GB RAM, 8 cores
https://github.com/torognes/vsearch

Reading file /tmp/qiime2-archive-ewumxl2w/327da406-e7db-4898-931f-df01072fcefd/data/dna-sequences.fasta 100%
303450669 nt in 213319 seqs, min 46, max 5604, avg 1423
minseqlength 32: 1 sequence discarded.
Masking 100%
Counting k-mers 100%
Creating k-mer index 100%
Searching 100%
Matching query sequences: 373 of 464 (80.39%)


Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: vsearch --usearch_global /tmp/qiime2-archive-6ri4ri5f/d7bb1a41-d3e8-465f-be11-90b58e1cf211/data/dna-sequences.fasta --id 0.8 --query_cov 0.8 --strand both --maxaccepts 10 --maxrejects 0 --db /tmp/qiime2-archive-4cipr0id/5a4b411f-26bc-4a88-9a05-7499d740c944/data/dna-sequences.fasta --threads 4 --output_no_hits --blast6out /tmp/tmpbso9jlee



vsearch v2.7.0_linux_x86_64, 15.5GB RAM, 8 cores
https://github.com/torognes/vsearch

Reading file /tmp/qiime2-archive-4cipr0id/5a4b411f-26bc-4a88-9a05-7499d740c944/data/dna-sequences.fasta 100%
88535911 nt in 322907 seqs, min 54, max 2366, avg 274
Masking 100%
Counting k-mers 100%
Creating k-mer index 100%
Searching 100%
Matching query sequences: 127 of 464 (27.37%)


## Save each of the taxonomy annotations for your sequences.

Finally, save the results of the vsearch taxonomy mapping into taxonomy .qza objects that you can use with downstream QIIME2 scripts. Note that these are *study-specific* classifications of your sequences, unlike the reference taxonomies, which are general.

In [52]:
for reference in vsearch_results:
    classification_taxonomy, = vsearch_results[reference]
    
    #Create a .qza file to hold the results of applying a given reference taxonomy
    #to your specific sequences (as distinct from the *reference* taxonomy for all known species)
    output_filepath = join(working_dir,'output',f"{reference}_classification_taxonomy.qza")
    classification_taxonomy.save(output_filepath)

# Remove mitochondria from samples

Now that we have created study-specific taxonomic annotations in the output folder (e.g. `../output/silva_extended_classification_taxonomy.qza`), we can use them to filter our feature tables to remove mitochondria or chloroplast 16S sequences.

## Set up filepaths

First we'll set up variables to hold the filepaths we'll need. 

In [61]:
#already done up top -- I think this is a duplicated cell; variables aren't called below

# output_dir = abspath("../output")
# input_filepath = abspath("../input")
#mapping_file = metadata_path
#sequence_file = seqs_path


#These files are specific to your study
feature_table_name = 'feature_table_live_vs_dead.qza'
feature_table_path = join(input_dir, feature_table_name)



#Load the taxonomy files created in the last section.
taxonomy_files = {"silva_base": "../output/silva_reference_taxonomy.qza",\
                  "silva_extended": "../output/silva_extended_reference_taxonomy.qza"}

required_files = [feature_table,mapping_file,sequence_file]

### Generate filtered feature tables tables with organelle sequences removed.

Next we'll use the QIIME2 filter_table command to remove features that are annotated as mitochondria or chloroplasts according to each taxonomy, and output filtered feature tables and feature table summaries for each to allow comparison and reporting.

In [ ]:
#not sure what's going on below w/r/t filtered_table = filter_table_results.filtered_table.
#is this equivalent to filtered_table, = filter_table_results?

#oh, yeah, just read the comment. Is one preferable over the other?


#how to approach naming output files?

In [62]:
filtered_feature_tables_by_taxonomy = defaultdict(dict)

feature_table = Artifact.load(feature_table_path)

for label, taxonomy_file in taxonomy_files.items():
    print(f"Analyzing data using the {label} taxonomy ({taxonomy_file})")
    taxonomy = Artifact.load(taxonomy_file)
    print(f"Removing mitochondia from: {feature_table}")

    #Remove organelle sequeces from the feature table
    #Note that the Qiime2 API does not return a single object, 
    #but rather a named Tuple struture with each output, which we save in filter_table_results
    filter_table_results = filter_table(feature_table,taxonomy,exclude="mitochondria,chloroplast",mode="contains")
    
    #Additionally remove any samples not in the metadata
    filter_table_results = filter_samples(filter_table_results.filtered_table,metadata=metadata)
    filtered_table = filter_table_results.filtered_table
    
    #Save the filtered feature table to a .qza file in the output directory
    output_filename = f"feature_table_filtered_{label}_mws.qza"
    output_filepath = join(output_dir,output_filename)
    print(f"Saving results to: {output_filepath}")
    filtered_table.save(output_filepath)
    
    #Output a feature table summary visualization
    summary_visualization = summarize(filtered_table,sample_metadata=metadata)
    vis = summary_visualization.visualization
    output_filename = f"feature_table_filtered_{label}_mws.qzv"
    output_filepath = join(output_dir,output_filename)
    print(f"Saving file summary to: {output_filepath}")
    
    filtered_feature_tables_by_taxonomy[label]=filtered_table
    print(f"Done with processing {label} taxonomy annotations!\n\n")

Analyzing data using the greengenes_reference taxonomy (../output/gg_reference_taxonomy.qza)
Removing mitochondia from: <artifact: FeatureTable[Frequency] uuid: 97994d43-af84-4dcd-a4a5-316bcc00a472>
Saving results to: /home/tanya/Work_files/organelle_removal/output/filtered_tables/feature_table_filtered_greengenes_reference_mws.qza
Saving file summary to: /home/tanya/Work_files/organelle_removal/output/filtered_tables/feature_table_filtered_greengenes_reference_mws.qzv
Done with processing greengenes_reference taxonomy annotations!


Analyzing data using the greengenes_extended taxonomy (../output/gg_extended_reference_taxonomy.qza)
Removing mitochondia from: <artifact: FeatureTable[Frequency] uuid: 97994d43-af84-4dcd-a4a5-316bcc00a472>
Saving results to: /home/tanya/Work_files/organelle_removal/output/filtered_tables/feature_table_filtered_greengenes_extended_mws.qza
Saving file summary to: /home/tanya/Work_files/organelle_removal/output/filtered_tables/feature_table_filtered_greengen

In [ ]:
filtered_feature_tables_by_taxonomy = defaultdict(dict)

feature_table = Artifact.load(feature_table_path)

for reference, classification_taxonomy in vsearch_results.items():
    print(f'Analyzing data using the {reference} classification taxonomy')
    print(f'Removing mitochondria from: {feature_table_name}')
    
    #Remove organelle sequeces from the feature table
    #Note that the Qiime2 API does not return a single object, 
    #but rather a named Tuple struture with each output, which we save in filter_table_results
    filter_table_results = filter_table(feature_table, classification_taxonomy,
                                        exclude = 'mitochondria,chloroplast',
                                        mode = 'contains')
    #Additionally remove any samples not in the metadata
    filter_table_results = filter_samples(filter_table_results.filtered_table,
                                          metadata = metadata)
    filtered_table = filter_table_results.filtered_table
    
    #Save the filtered feature table to a .qza file in the output directory
    filtered_table_path = join(output_dir, f'feature_table_filtered_{reference}.qza')
    print(f'Saving results to: {filtered_table_path}')
    filtered_table.save(filtered_table_path)
    
    #output a feature table summary visualization
    summary_visualization = summarize(filtered_table, sample_metadata = metadata)
    vis = summary_visualization.visualization
    visualization_path = join(output_dir, f'feature_table_filtered_{reference}.qzv')
    print(f'Saving file summary to: {visualization_path}')
    vis.save(visualization_path)
    
    
    

# Rarefy tables to an even depth

In [68]:
#choose a rarefaction depth appropriate for your study.
rarefaction_depth = 1000

rarefied_feature_tables_by_taxonomy = defaultdict(dict)

for label,filtered_feature_tables in filtered_feature_tables_by_taxonomy.items():
    print(f"Rarefying feature table {filtered_feature_tables} to {rarefaction_depth} sequences/sample.")
    rarefy_results = rarefy(table=filtered_feature_tables,sampling_depth=rarefaction_depth)
    #get the rarefied table out of the NamedTuple of results
    rarefied_filtered_table = rarefy_results.rarefied_table
        
    #save the resulting feature table
    output_filename = f"feature_table_{label}_{rarefaction_depth}.qza"
    output_filepath = join(output_dir,output_filename)
    print("Saving results to:{output_filepath}")
    rarefied_filtered_table.save(output_filepath)
        
    #store the rarefied tables in a dict so they don't need to relode them
    rarefied_feature_tables_by_taxonomy[label] = rarefied_filtered_table

Rarefying feature table <artifact: FeatureTable[Frequency] uuid: 44d128c1-3928-45c1-812b-5b4951168af0> to 1000 sequences/sample.
Saving results to:{output_filepath}
Rarefying feature table <artifact: FeatureTable[Frequency] uuid: 198b8748-a810-412f-b31a-ec9bbdd5def8> to 1000 sequences/sample.
Saving results to:{output_filepath}
Rarefying feature table <artifact: FeatureTable[Frequency] uuid: c6323e2b-0a78-4e72-939e-08cc227a6085> to 1000 sequences/sample.
Saving results to:{output_filepath}
Rarefying feature table <artifact: FeatureTable[Frequency] uuid: 5bb410e1-75b4-41c1-8887-073b4db4ac1e> to 1000 sequences/sample.
Saving results to:{output_filepath}
